In [1]:
import random
import time

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from torch import rand

In [2]:
def distance(x1,y1,x2,y2):
    return np.sqrt((x2-x1)**2+(y2-y1)**2)

In [3]:
from queue import PriorityQueue
def dijkstra(n,graph,start_vertex,weight):
    D = [float('inf') for v in range(n)]
    D[start_vertex] = 0
    visited=[]

    pq = PriorityQueue()
    pq.put((0, start_vertex))

    while not pq.empty():
        (dist, current_vertex) = pq.get()
        visited.append(current_vertex)

        for neighbor in range(n):
            if weight[current_vertex][neighbor] != -1:
                distance = weight[current_vertex][neighbor]
                if neighbor not in visited:
                    old_cost = D[neighbor]
                    new_cost = D[current_vertex] + distance
                    if new_cost < old_cost:
                        pq.put((new_cost, neighbor))
                        D[neighbor] = new_cost
    return D

In [11]:
def dataset(num):
    Max=[[1e-7,0,0] for i in range(num)]

    for h in range(num):
        n=random.randint(5,20)
        X=rand(n,2)
        X=X.numpy()
        W=np.zeros((n,n))

        for i in range(n):
            for j in range(i+1,n):
                W[i][j]=W[j][i]=distance(X[i][0],X[i][1],X[j][0],X[j][1])
        Max[h][2]=W
        Max[h][2]=np.pad(Max[h][2],(20-n)//2,mode='constant')
        if(n%2!=0):
            Max[h][2]=np.pad(Max[h][2],[(0,1),(0,1)],mode='constant')

        for i in range(100):
            for k in range(2,5):
                t1=time.time()
                G = nx.watts_strogatz_graph(n = n, k = k,p=random.random())
                # pos=dict(zip(G,X))
                t2=time.time()
                spl=[]

                for i in range(n):
                    D=dijkstra(n,G,i,W)
                    spl.append(D)
                    
                aplSum=0.
                for i in range(n):
                    aplSum+=np.sum(spl[i])
                apl=aplSum/(n*(n-1))
                acc=nx.average_clustering(G)
                adj=nx.adjacency_matrix(G).toarray()*1.
                adj=W*np.transpose(adj)
                adj=np.pad(adj,(20-n)//2,mode='constant')

                if(n%2!=0):
                    adj=np.pad(adj,[(0,1),(0,1)],mode='constant')

                if(t2-t1>0):
                    v=acc/(apl*(t2-t1))
                    if(v>=Max[h][0]):
                        Max[h][0]=v
                        Max[h][1]=adj
    return Max

In [5]:
# f=dataset(3)

# from PIL import Image
# im=Image.fromarray(f[0][1],'RGB')
# im.show()

In [12]:
def dataset_txt(MAX,num):
    f1=open('Dataset3.txt','r+')
    f1.truncate(0)
    f2=open('Dataset4.txt','r+')
    f2.truncate(0)
    for h in range(num):
        with open('Dataset3.txt','a') as f1:
            for line in np.matrix(MAX[h][2]):
                np.savetxt(f1,line,fmt='%.2f')
            f1.close()
        with open('Dataset4.txt','a') as f2:
            for line in np.matrix(MAX[h][1]):
                np.savetxt(f2,line,fmt='%.2f')
            f2.close()

In [17]:
d=dataset(2000)
dataset_txt(d,2000)